In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict, load_dataset
import torch
import random

c:\Users\WONJONGHYEON\anaconda3\envs\CP2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [543]:
df=pd.read_csv('train.csv').drop(['ID'], axis=1)
trans_aug=pd.read_csv(r'C:\Users\WONJONGHYEON\OneDrive\바탕 화면\cp2-competition\augment_csv\trans_aug.csv').drop(['label'], axis=1)

trans_aug['label'] = trans_aug['유형'] + '-' + trans_aug['극성'] + '-' + trans_aug['시제'] + '-' + trans_aug['확실성']

In [545]:
label_table = df['label'].value_counts().to_frame()
label_table.head()

,label
사실형-긍정-과거-확실,7113
사실형-긍정-현재-확실,4743
추론형-긍정-현재-확실,1101
사실형-긍정-미래-확실,649
추론형-긍정-과거-확실,335


In [546]:
aug_label_table = trans_aug['label'].value_counts().to_frame()
aug_label_table.head()

,label
예측형-긍정-미래-불확실,263
예측형-긍정-미래-확실,107
대화형-미정-현재-불확실,47
사실형-미정-미래-확실,42
사실형-미정-미래-불확실,38


In [ ]:
aug_label_table

In [547]:
labels = label_table[label_table.label>=10].index

In [548]:
aug_labels = aug_label_table[aug_label_table.label>=10].index

In [549]:
aug_labels

Index(['예측형-긍정-미래-불확실', '예측형-긍정-미래-확실', '대화형-미정-현재-불확실', '사실형-미정-미래-확실',
       '사실형-미정-미래-불확실', '대화형-미정-미래-불확실', '추론형-미정-미래-불확실', '사실형-미정-현재-확실',
       '예측형-긍정-현재-확실', '예측형-미정-미래-불확실', '예측형-긍정-과거-확실', '예측형-긍정-현재-불확실',
       '예측형-긍정-과거-불확실', '대화형-미정-과거-불확실'],
      dtype='object')

In [550]:
idx_lst, aug_idx_lst = [], []

In [551]:
for idx, label in enumerate(df['label']):
    if not label in list(labels):
        idx_lst.append(idx)

for idx, label in enumerate(trans_aug['label']):
    if not label in list(aug_labels):
        aug_idx_lst.append(idx)

In [552]:
df_in=df.drop(idx_lst)
df_in.reset_index(drop=True, inplace=True)

df_out=df.iloc[idx_lst]
df_out.reset_index(drop=True, inplace=True)
df_out=df_out.sample(frac=1)


In [553]:
aug_in=trans_aug.drop(aug_idx_lst)
aug_in.reset_index(drop=True, inplace=True)

aug_out=trans_aug.iloc[aug_idx_lst]
aug_out.reset_index(drop=True, inplace=True)
aug_out=aug_out.sample(frac=1)

In [570]:
from sklearn.model_selection import train_test_split
df_t, df_v, y_t, y_v = train_test_split(df_in[['문장', '유형', '극성', '시제', '확실성']], df_in[['label']], test_size=0.2, stratify=df_in['label'], random_state=1)
aug_t, aug_v, aug_y_t, aug_y_v = train_test_split(aug_in[['문장', '유형', '극성', '시제', '확실성']], aug_in[['label']], test_size=0.2, stratify=aug_in['label'], random_state=1)

In [573]:
aug_y_t.value_counts(normalize=True)

label        
예측형-긍정-미래-불확실    0.375000
예측형-긍정-미래-확실     0.151786
대화형-미정-현재-불확실    0.067857
사실형-미정-미래-확실     0.060714
대화형-미정-미래-불확실    0.053571
사실형-미정-미래-불확실    0.053571
추론형-미정-미래-불확실    0.046429
사실형-미정-현재-확실     0.037500
예측형-긍정-현재-확실     0.032143
예측형-미정-미래-불확실    0.030357
예측형-긍정-과거-확실     0.025000
예측형-긍정-현재-불확실    0.023214
대화형-미정-과거-불확실    0.021429
예측형-긍정-과거-불확실    0.021429
dtype: float64

In [572]:
aug_y_v.value_counts(normalize=True)

label        
예측형-긍정-미래-불확실    0.375887
예측형-긍정-미래-확실     0.156028
대화형-미정-현재-불확실    0.063830
사실형-미정-미래-불확실    0.056738
사실형-미정-미래-확실     0.056738
대화형-미정-미래-불확실    0.049645
추론형-미정-미래-불확실    0.049645
사실형-미정-현재-확실     0.035461
예측형-긍정-현재-확실     0.035461
예측형-긍정-과거-확실     0.028369
예측형-미정-미래-불확실    0.028369
대화형-미정-과거-불확실    0.021277
예측형-긍정-과거-불확실    0.021277
예측형-긍정-현재-불확실    0.021277
dtype: float64

In [555]:
len(df_out), len(aug_out)

(92, 52)

In [556]:
df_t = pd.concat([df_t, df_out[:41].drop(['label'], axis=1)], axis=0)
df_v = pd.concat([df_v, df_out[41:].drop(['label'], axis=1)], axis=0)
aug_t = pd.concat([aug_t, aug_out[:26].drop(['label'], axis=1)], axis=0)
aug_v = pd.concat([aug_v, aug_out[26:].drop(['label'], axis=1)], axis=0)

In [563]:
df_t = pd.concat([df_t, aug_t], axis=0)
df_t.drop_duplicates(inplace=True)
df_v = pd.concat([df_v, aug_v], axis=0)
df_v.drop_duplicates(inplace=True)

In [564]:
df_t

,문장,유형,극성,시제,확실성
3680,특히 거래량 중 고래들이 차지하는 비중이 높을 때 더욱 그렇다.,사실형,긍정,현재,확실
8405,그러나 닛산 측은 모든 권한이 곤 회장에게 과도하게 집중돼 그가 전횡을 일삼은 것이...,사실형,긍정,과거,확실
7944,서울의 한 아파트에 사는 A씨는 윗집에 사는 B씨에게 층간소음 문제로 항의하다 결국...,사실형,긍정,과거,확실
266,하지만 현행 동물 보호법은 ＇목을 매달면 안 된다＇든가 ＇공개된 장소에서 죽이면 안...,사실형,긍정,현재,확실
7185,허리는 묶은 듯이 오목하고 빛깔은 말린 생강 같다.,추론형,긍정,현재,확실
...,...,...,...,...,...
39,"공군의 태평양 횡단 직항 비행은 처음으로, 이번 방미 비행을 통해 조종사들은 장거리...",추론형,미정,현재,확실
10,"""당신이 처음부터 OTT 사업을 성장시키고 싶었다면, Cupplay의 'Time t...",예측형,부정,과거,확실
50,처음으로 공군의 교차 태평양 직항 비행은 조종사가 장거리 항해 훈련을 경험할 수있게...,추론형,미정,현재,확실
28,정 대표는 ＂세계적으로 사망률이 높은 만성질환인 뇌졸중에 대한 예방제 외에 삶의 질...,사실형,미정,현재,불확실


In [565]:
df_v

,문장,유형,극성,시제,확실성
3048,"앞서 한 장관은 현 경제 상황에 대해 ＂경제전쟁이 시작됐다고 할 만큼 물가 상승, ...",사실형,긍정,과거,확실
5017,전통 유통업의 몰락은 비단 한국만의 현상은 아니다.,사실형,부정,현재,확실
16031,이들은 7차례에 걸쳐 OLED 패널 대형화 핵심 기술을 LG디스플레이 측에 넘겼다.,사실형,긍정,과거,확실
918,4년 전 손목 부상과 슬럼프에 빠졌던 위기를 넘기고 누구도 예상하지 못한 금메달을 ...,추론형,긍정,과거,확실
14363,이 부회장은 ＂서로에게 기생하고 있다는 것은 모든 사람에게 영향을 미치는 것＂이라며...,사실형,긍정,과거,확실
...,...,...,...,...,...
41,그러나 최근에는 표적 치료의 출현이 전염된 유방암 치료의 효율성을 크게 향상시켰으며...,사실형,미정,현재,불확실
14,중국은 새로운 코로나 바이러스 감염에 대처하고 내부 물을 정상화하기 위해 미국에 약...,예측형,부정,미래,불확실
13,"Baugh 교수는 ""국내 사업이라면 LP 자회사 부동산 시장이 완전히 운영되고 세금...",예측형,미정,현재,불확실
46,그는 “전통적인 뇌졸중에 대한 예방 조치 외에도 전 세계적으로 높은 사망률이 있으며...,사실형,미정,현재,불확실


In [566]:
df_t['유형'].value_counts(), df_v['유형'].value_counts()

(사실형    10874
 추론형     1724
 대화형      498
 예측형      442
 Name: 유형, dtype: int64,
 사실형    2740
 추론형     451
 예측형     155
 대화형     140
 Name: 유형, dtype: int64)

In [567]:
df_t['극성'].value_counts(), df_v['극성'].value_counts()

(긍정    12834
 부정      439
 미정      265
 Name: 극성, dtype: int64,
 긍정    3244
 부정     135
 미정     107
 Name: 극성, dtype: int64)

In [568]:
df_t['시제'].value_counts(), df_v['시제'].value_counts()

(과거    6433
 현재    5531
 미래    1574
 Name: 시제, dtype: int64,
 과거    1625
 현재    1415
 미래     446
 Name: 시제, dtype: int64)

In [569]:
df_t['확실성'].value_counts(), df_v['확실성'].value_counts()

(확실     12235
 불확실     1303
 Name: 확실성, dtype: int64,
 확실     3095
 불확실     391
 Name: 확실성, dtype: int64)

In [575]:
import re

def preprocess(df): 
  df['문장']=df['문장'].map(lambda x: re.sub(r'[\d]+', '1', x)) #숫자를 1로 바꿈.
  df['문장']=df['문장'].map(lambda x: re.sub(r'[a-zA-Z]+', 'a', x)) #영어를 a로 바꿈.
  df['문장']=df['문장'].map(lambda x: re.sub(r'[^ a가-힣1]', '', x)) #한글, 1, a 제외 문자 제거

In [576]:
preprocess(df_t)
preprocess(df_v)

(None, None)

In [383]:
df_aug = pd.concat([df_aug, trans_aug], axis=0).drop_duplicates()

In [384]:
df_aug

,문장,유형,극성,시제,확실성
12,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 떨어지는 ...,2,0,1,0
39,사람 친구는 어느 한쪽에 애인이 생기면 끊어주는 게 쌈빡한 예의 아닐까,0,0,1,1
47,a 사업을 처음부터 직접 키우려 했다면 쿠팡플레이의 타임 투 마켓은 1년이 걸렸을 것이다,2,2,0,1
63,산소통도 셰르파도 포기한 채 나만의 힘으로 오른다는 뜻입니다,0,0,2,1
72,최민철 배우의 a형 독감 확진 판정으로 뮤지컬 레베카 1월 1일 잭 파벨 역 캐스팅...,0,0,0,1
...,...,...,...,...,...
748,시베리아 전염병과 마찬가지로 우리가 이미 알고있는 전염병이나 새로운 감염은 동물 전...,0,1,1,0
749,새로운 a a a a 1는 기존의 a 1 월드 챔피언십 시리즈의 후계자로 1 개 이...,1,1,1,1
750,거기 거리가 있나요,0,1,2,0
751,aa의 운영 대상인 공군은 1월 1일부터 1월 1일까지 한국 유형 전투기 aa 지명...,1,1,1,1


In [385]:
df_aug.유형.value_counts()

1    1737
3     732
0     611
2     484
Name: 유형, dtype: int64

In [387]:
df_aug=pd.concat([df_aug, df[df['유형']==1].sample(n=200, random_state=1)]) #증강할 문장: 유형-사실 200
df_aug=pd.concat([df_aug, df[df['유형']==3].sample(n=800, random_state=3)]) #증강할 문장: 유형-추론 800
df_aug=pd.concat([df_aug, df[df['극성']==0].sample(n=200, random_state=1)]) #증강할 문장: 극성-긍정 200
df_aug=pd.concat([df_aug, df[df['확실성']==1].sample(n=200, random_state=2)]) #증강할 문장: 확실성-확실 200
df_aug=pd.concat([df_aug, df[df['시제']!=1].sample(n=200, random_state=2)]) #증강할 문장: 시제-현재, 과거 200

In [389]:
df_aug = df_aug.drop_duplicates()

In [390]:
df_aug.유형.value_counts(), df_aug.극성.value_counts()

(1    2356
 3    1311
 0     611
 2     484
 Name: 유형, dtype: int64,
 0    3892
 2     569
 1     301
 Name: 극성, dtype: int64)

In [391]:
df_aug.시제.value_counts(), df_aug.확실성.value_counts()

(1    1889
 2    1720
 0    1153
 Name: 시제, dtype: int64,
 1    3192
 0    1570
 Name: 확실성, dtype: int64)

In [95]:
import spacy
from spacy.lang.ko.examples import sentences 
nlp = spacy.load("ko_core_news_sm") #python -m spacy download ko_core_news_sm

In [96]:
# open 불용어.txt, stop_words_set(불용어 사전) 만듬.
import locale

# 지역 설정 변경
locale.setlocale(locale.LC_COLLATE, 'ko_KR.UTF-8')

stop_words = {}

with open('불용어.txt', 'r') as f:
    for line in f:
        line = line.strip('\n')
        line = line.split(':')  # 줄 바꿈 문자 제거
        key = line[0] #key
        value = line[1].split(', ')  # value
        value.sort(key=locale.strxfrm)
        stop_words[key] = value  # 딕셔너리에 추가


# stop_words_set
stop_words_lst=[]
for _, value in stop_words.items():
    stop_words_lst.extend(value)

stop_words_set=set(stop_words_lst)
stop_words_set.update(nlp.Defaults.stop_words)

In [97]:
# 특수문자, 괄호, 괄호 안 문장 제거 함수 (증강 함수에 포함됨)
def get_only_words(line):
  parseText= re.compile(r'\([^()]*\)').sub('',  line)
  parseText= re.compile('[^ \w]').sub('', parseText)

  return parseText

In [ ]:
neg_adv_lst=['못', '안', '전혀', '결코', '별로', '그리', '도대체', '도저히', '절대', '그다지', '절대로', '도무지', '채', '미처', '좀처럼', '여간', '영', '비단', '차마', '통', '구태여']

In [137]:
for neg_adv in neg_adv_lst:
    if neg_adv in stop_words_set:
        stop_words_set.remove(neg_adv)

In [161]:
'수' in stop_words_set

True

In [162]:
stop_words_set.remove('수') #예측에 많이 쓰이는 의존명사

In [163]:
def random_deletion(words, p):

    new_words=[]
    for token in words:
        r=random.uniform(0, 1)
        if r>p:
            if token.text in stop_words_set: #1-p의 확률로 불용어를 제거한다.
                continue
            if (token.pos_ =='ADV') and (token.dep_ not in ['ROOT', 'fixed']) and (token.text not in neg_adv_lst): #1-p의 확률로 부정 부사를 제외한 부사 제거
                continue
            if token.pos_ in ['ADJ', 'NUM'] and token.dep_ not in ['ROOT', 'fixed']: #1-p확률로 수식하는 단어를 제거
                continue
        
        new_words.append(token.text) #포함되는 토큰을 담는다.
    return new_words

In [412]:
doc = nlp("제1호 태풍 마이삭이 오는 이상의 전국에 바람과 일부 지역에는 11일 물폭탄이 예상된다")
for token in doc:
    print(token.text, token.pos_, token.dep_)

제1호 PART nmod
태풍 NOUN compound
마이삭이 NOUN nsubj
오는 VERB acl
이상의 NOUN nmod
전국에 ADV obl
바람과 CCONJ dislocated
일부 NOUN compound
지역에는 NOUN conj
11일 NOUN compound
물폭탄이 NOUN dislocated
예상된다 ADV ROOT


In [148]:
random_deletion(doc, 0)


['마이크', '테스트', '전혀', '모르겠다', '.']

In [149]:
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	if len(new_words) > 4:
		random_idx_1 = random.randint(1, len(new_words)-2) #양 끝 단어는 교체하지 못하게 함.
		random_idx_2 = random_idx_1
		counter = 0

		while random_idx_2 == random_idx_1:
			random_idx_2 = random.randint(1, len(new_words)-2)
			counter += 1
			if counter > 3:
				return new_words

		new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words


In [167]:
def EDA(sentence, alpha_rs=0.1, p_rd=0.5, num_aug=5): #증강 문장 수, 제거 확률은 하이퍼파라미터처럼 조절해야 할 듯 함.

	sentence = get_only_words(sentence)
	docs = nlp(sentence)

	augmented_sentences = []

	# rd
	for _ in range(num_aug):
		a_words = random_deletion(docs, p_rd)
		if len(a_words) > 4:
			augmented_sentences.append(" ".join(a_words))
	
	if len(augmented_sentences) < 2:
		augmented_sentences.append(sentence)

	
	# rs
	for i, sent in enumerate(augmented_sentences):
		words = sent.split(' ')
		words = [word for word in words if word != ""]
		if words:
			if len(words)<20:
				num_words=len(words)
				n_rs = max(1, int(alpha_rs*num_words))
				a_words = random_swap(words, n_rs)
				augmented_sentences[i] = " ".join(a_words)

	random.shuffle(augmented_sentences)


	return augmented_sentences

In [ ]:
doc = nlp("비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 떨어지는 곳이 있을 예정이다")
for token in doc:
    print(token.text, token.pos_, token.dep_)

In [ ]:
EDA("비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 떨어지는 곳이 예정이다")

In [308]:
df_aug.reset_index(drop=True, inplace=True)

In [314]:
df_aug

,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 떨어지는 ...,2,0,1,0
1,사람 친구는 어느 한쪽에 애인이 생기면 끊어주는 게 쌈빡한 예의 아닐까,0,0,1,1
2,a 사업을 처음부터 직접 키우려 했다면 쿠팡플레이의 타임 투 마켓은 1년이 걸렸을 것이다,2,2,0,1
3,산소통도 셰르파도 포기한 채 나만의 힘으로 오른다는 뜻입니다,0,0,2,1
4,최민철 배우의 a형 독감 확진 판정으로 뮤지컬 레베카 1월 1일 잭 파벨 역 캐스팅...,0,0,0,1
...,...,...,...,...,...
4357,고대일록은 리더십의 실종이 어떤 결과를 초래하는지 잘 보여준다,3,0,2,1
4358,조선의 지배층은 길이 넓고 다리가 견고할 경우 조선이 잦은 외침을 받아 남아나지 않...,1,0,0,1
4359,그러나 이 같은 수치는 한 달 새 코픽스의 변동을 단순 계산한 것이며 이번달 코픽스...,3,0,2,1
4360,그러나 여전히 11는 주위 사람들의 시선이 부담스러워 구매를 못 했다고 응답했다,1,0,0,1


In [344]:
def rs_rd_aug(df):
  df_new=df.copy()
  aug_sent_lst = []
  label_lst = []

  for idx, sent in enumerate(df_new['문장']):
    aug_sents = EDA(sent)
    aug_num = len(aug_sents)
    label=list(df_new[['유형', '극성', '시제', '확실성']].iloc[idx].values)
    
    sent_label_lst = [label for _ in range(aug_num)]
    label_lst.extend(sent_label_lst)

    aug_sent_lst.extend(aug_sents)
    
  label_df = pd.DataFrame(label_lst, columns=['유형', '극성', '시제', '확실성'], dtype=int)
  sent_df = pd.DataFrame(aug_sent_lst, columns=['문장'])
  
  aug_df=pd.concat([sent_df, label_df], axis = 1)

  return aug_df

In [327]:
df_aug

,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 떨어지는 ...,2,0,1,0
1,사람 친구는 어느 한쪽에 애인이 생기면 끊어주는 게 쌈빡한 예의 아닐까,0,0,1,1
2,a 사업을 처음부터 직접 키우려 했다면 쿠팡플레이의 타임 투 마켓은 1년이 걸렸을 것이다,2,2,0,1
3,산소통도 셰르파도 포기한 채 나만의 힘으로 오른다는 뜻입니다,0,0,2,1
4,최민철 배우의 a형 독감 확진 판정으로 뮤지컬 레베카 1월 1일 잭 파벨 역 캐스팅...,0,0,0,1
...,...,...,...,...,...
4357,고대일록은 리더십의 실종이 어떤 결과를 초래하는지 잘 보여준다,3,0,2,1
4358,조선의 지배층은 길이 넓고 다리가 견고할 경우 조선이 잦은 외침을 받아 남아나지 않...,1,0,0,1
4359,그러나 이 같은 수치는 한 달 새 코픽스의 변동을 단순 계산한 것이며 이번달 코픽스...,3,0,2,1
4360,그러나 여전히 11는 주위 사람들의 시선이 부담스러워 구매를 못 했다고 응답했다,1,0,0,1


In [392]:
df_aug_ = rs_rd_aug(df_aug)

In [393]:
df_aug_

,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 치고 곳이 지역에선 우박이 떨어지는 일부 있을 예정이다,2,0,1,0
1,비가 떨어지는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 내리는 ...,2,0,1,0
2,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 우박이 있을 곳이 떨어지는...,2,0,1,0
3,비가 곳이 지역에서는 천둥번개가 치고 일부 우박이 떨어지는 내리는 예정이다,2,0,1,0
4,비가 내리는 곳이 돌풍과 치고 일부 지역에선 우박이 떨어지는 지역에서는 예정이다,2,0,1,0
...,...,...,...,...,...
22587,각 인물들은 모두 사연과 이유로 하나뿐인 모습을 가방을 차지하려 하는 돈 보인다,1,0,2,1
22588,각 인물들은 다른 하는 이유로 하나뿐인 돈 가방을 차지하려 사연과 모습을 보인다,1,0,2,1
22589,각 인물들은 모두 다른 하나뿐인 이유로 사연과 돈 가방을 차지하려 하는 모습을 보인다,1,0,2,1
22590,각 인물들은 모두 다른 사연과 가방을 단 돈 이유로 차지하려 하는 모습을 보인다,1,0,2,1


In [394]:
df_aug_.drop_duplicates(inplace=True)

In [395]:
df_aug_.isna().sum()

문장     0
유형     0
극성     0
시제     0
확실성    0
dtype: int64

In [396]:
df_aug_.유형.value_counts(), df_aug_.극성.value_counts()

(1    10846
 3     5984
 0     2483
 2     2303
 Name: 유형, dtype: int64,
 0    17785
 2     2536
 1     1295
 Name: 극성, dtype: int64)

In [364]:
df_aug.시제.value_counts()

1    1889
2    1421
0    1052
Name: 시제, dtype: int64

In [397]:
df_aug_.시제.value_counts(), df_aug_.확실성.value_counts()

(1    8900
 2    7479
 0    5237
 Name: 시제, dtype: int64,
 1    14378
 0     7238
 Name: 확실성, dtype: int64)

In [398]:
df_type = df[df['유형']!=1]
df_polar = df[df['극성']!=0]
df_certainty = df[df['확실성']==0]
df_tense = df[df['시제']==1]

In [400]:
df_under_type = df[df['유형']==1].sample(2000)
df_under_polar = df[df['극성']==0].sample(2000)
df_under_certainty = df[df['확실성']==1].sample(2000)
df_under_tense = df[df['시제']!=1].sample(2000)

In [402]:
df_under = pd.concat([df_under_type, df_under_polar, df_under_certainty, df_under_tense]).drop_duplicates()

In [399]:
df_aug_=pd.concat([df_aug_, df_type, df_polar, df_certainty, df_tense]).drop_duplicates()


,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 치고 곳이 지역에선 우박이 떨어지는 일부 있을 예정이다,2,0,1,0
1,비가 떨어지는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 내리는 ...,2,0,1,0
2,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 우박이 있을 곳이 떨어지는...,2,0,1,0
3,비가 곳이 지역에서는 천둥번개가 치고 일부 우박이 떨어지는 내리는 예정이다,2,0,1,0
4,비가 내리는 곳이 돌풍과 치고 일부 지역에선 우박이 떨어지는 지역에서는 예정이다,2,0,1,0
...,...,...,...,...,...
16286,산재로 인해 노동자가 사망할 경우 도급인에게는 1년 이하 징역 또는 1억원 이하 벌...,1,0,1,1
16311,장중 대규모 주식을 매도하면 주가가 급락할 수 있어 장이 끝난 뒤 매매에 나선다,1,0,1,1
16340,중국 터키 스웨덴 등은 조만간 시범운영에 나설 예정이다,1,0,1,1
16470,채권 만기는 내년 1월로 이를 상환하지 못하면 디폴트 위기를 맞게 된다,1,0,1,1


In [403]:
pd.concat([df_aug_, df_under]).drop_duplicates()

,문장,유형,극성,시제,확실성
0,비가 내리는 지역에서는 치고 곳이 지역에선 우박이 떨어지는 일부 있을 예정이다,2,0,1,0
1,비가 떨어지는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 지역에선 우박이 내리는 ...,2,0,1,0
2,비가 내리는 지역에서는 돌풍과 함께 천둥번개가 치고 일부 우박이 있을 곳이 떨어지는...,2,0,1,0
3,비가 곳이 지역에서는 천둥번개가 치고 일부 우박이 떨어지는 내리는 예정이다,2,0,1,0
4,비가 내리는 곳이 돌풍과 치고 일부 지역에선 우박이 떨어지는 지역에서는 예정이다,2,0,1,0
...,...,...,...,...,...
14005,또한 a씨가 자신이 근무한 편의점에서 1일 동안 최소 1만원 이상의 돈을 들여 상품...,1,0,0,0
3872,오유리 빗썸경제연구소 정책연구팀장은 국내 규제가 미국의 동향을 따를 가능성이 높은 ...,1,0,0,1
14875,보라네트워크는 카카오게임즈의 계열사다,1,0,2,1
5136,컴투스는 최근 크리티카 글로벌의 a1a 블록체인 생태계 합류를 위해 참여자들이 직접...,1,0,0,1


In [404]:
df_aug = pd.concat([df_aug_, df_under]).drop_duplicates()

In [406]:
len(df_aug)

28124

In [409]:
df_aug.확실성.value_counts()

1    20608
0     7516
Name: 확실성, dtype: int64

In [410]:
df_aug.to_csv('df_aug.csv', index=False, encoding='utf-8-sig')